In [15]:
# 필요한 라이브러리 설치
!pip install unidecode rapidfuzz

from google.colab import files
from rapidfuzz import fuzz
import re
import unidecode
import string

# 자막 파일에서 각 자막의 텍스트를 추출하는 함수
def extract_subtitles(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # 자막 블록을 빈 줄을 기준으로 분리
    blocks = content.strip().split('\n\n')
    subtitles = []

    for block in blocks:
        lines = block.strip().split('\n')
        if len(lines) >= 3:
            # 자막 번호와 타임스탬프를 제외한 나머지 라인들을 합침
            text = ' '.join(lines[2:])
            subtitles.append({'index': lines[0], 'text': text})
        elif len(lines) >= 2:
            text = lines[1]
            subtitles.append({'index': lines[0], 'text': text})
        else:
            continue

    return subtitles

# 텍스트 전처리 함수
def preprocess_text(text):
    # 대소문자 무시를 위해 소문자로 변환
    text = text.lower()
    # 유니코드 정규화 (특수 문자 통일)
    text = unidecode.unidecode(text)
    # 구두점 제거
    text = text.translate(str.maketrans('', '', string.punctuation))
    # 불필요한 공백 제거
    text = text.strip()
    return text

# N-그램 생성 함수
def generate_ngrams(text, n):
    words = text.split()
    ngrams = [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]
    return ngrams

# 제공한 고유명사 리스트
proper_nouns_list = ['Google', 'Netflix']

# 고유명사 리스트 전처리 (소문자 변환 및 구두점 제거)
processed_proper_nouns = [preprocess_text(noun) for noun in proper_nouns_list]

# 자막 파일 업로드
uploaded = files.upload()

# 업로드한 파일 이름 가져오기
uploaded_filename = list(uploaded.keys())[0]

# 자막 파일에서 각 자막 추출
subtitles = extract_subtitles(uploaded_filename)

# 결과 저장을 위한 리스트
results = []

# 유사도 임계값 설정 (0~100 사이의 값, 높을수록 엄격)
similarity_threshold = 80

for subtitle in subtitles:
    index = subtitle['index']
    original_text = subtitle['text']
    text = preprocess_text(original_text)
    found_nouns = []

    # 자막 텍스트의 단어 수
    text_words = text.split()
    text_length = len(text_words)

    for i, proper_noun in enumerate(processed_proper_nouns):
        # 고유명사의 단어 수
        proper_noun_length = len(proper_noun.split())

        # N-그램 생성
        ngrams = generate_ngrams(text, proper_noun_length)

        max_similarity = 0
        best_match = ''

        for ngram in ngrams:
            # 유사도 계산
            similarity = fuzz.ratio(proper_noun, ngram)
            if similarity > max_similarity:
                max_similarity = similarity
                best_match = ngram

        if max_similarity == 100:
            # 정확히 일치하는 경우
            found_nouns.append({'proper_noun': proper_nouns_list[i], 'status': '일치'})
        elif max_similarity >= similarity_threshold:
            # 유사도가 임계값 이상인 경우 오탈자 의심
            found_nouns.append({'proper_noun': proper_nouns_list[i], 'status': f"오탈자 의심 (유사도 {max_similarity}%, '{best_match}')"})

    if found_nouns:
        results.append({'index': index, 'text': original_text, 'found_nouns': found_nouns})

# 결과 출력
for res in results:
    print(f"\n자막 번호: {res['index']}")
    print(f"자막 내용: {res['text']}")
    for noun_info in res['found_nouns']:
        print(f"  고유명사 '{noun_info['proper_noun']}': {noun_info['status']}")


Saving 전과자_오타.srt to 전과자_오타 (3).srt

자막 번호: 10
자막 내용: [This is POSTECHS] KAIST...
  고유명사 'POSTECH': 오탈자 의심 (유사도 93.33333333333333%, 'postechs')

자막 번호: 13
자막 내용: [Changing Majors]
  고유명사 'Changing Majors': 일치

자막 번호: 14
자막 내용: [The city of steel, Sohang]
  고유명사 'Pohang': 오탈자 의심 (유사도 83.33333333333334%, 'sohang')

자막 번호: 19
자막 내용: [Bow] I'm Lee ChangSub, the Jeongwaja.
  고유명사 'Jeongwaja': 일치
  고유명사 'Lee ChangSub': 일치

자막 번호: 21
자막 내용: [Pohaha] It's Ppohang.
  고유명사 'Pohang': 오탈자 의심 (유사도 92.3076923076923%, 'ppohang')

자막 번호: 22
자막 내용: [Past memories in Pohang...] We've been here with the show 'WhoRide'.
  고유명사 'Pohang': 일치

자막 번호: 23
자막 내용: [WhoRide 2 Coming Soon in November] I didn’t know I’d find myself back in Pohang.
  고유명사 'Pohang': 일치

자막 번호: 31
자막 내용: [Forget the latecomer, let's start shooting] Alright, Pohang.
  고유명사 'Pohang': 일치

자막 번호: 32
자막 내용: [Turned into a Detective] There’s only one reason I’m in Pohang, right?
  고유명사 'Pohang': 일치

자막 번호: 34
자막 내용: [a.k.a POSTE